## Libraries

In [1]:
import pandas as pd
import re

## Open csv

In [2]:
df = pd.read_csv("../output/frenchCropUsage_20210525/d2kab/conjunctions.csv", delimiter="\t", encoding='utf-8')
df[:3]

,bsv,context before,conjunction,context after,location,features
0,20180911_bsv_grandes_cultures-26_cle0f15a8,1 Pluri-annuelles (,"prairies, jachères",) 5 Colza/Blé Maïs/,4028-4046,"type=N_PUN_N, form=prairies, jachères"
1,20181106_bsv_grandes_cultures_34_cle08159b,7 feuilles Enquête,Tournesol et oiseaux,Céréales à paille L,675-695,"type=N_KON_N, form=Tournesol et oiseaux"
2,20190320_LOR_BSV_Grandes_Cultures_cle83816d,n démarrée pour les,blés et orges,d'hiver Colza .....,244-257,"type=N_KON_N, form=blés et orges"


In [54]:
df = pd.read_csv("../output/frenchCropUsage_20210525/vespa/conjunctions.csv", delimiter="\t", encoding='utf-8')
df[:3]

,bsv,context before,conjunction,match,context after,location,features
0,04_bsv_raisin_12042012_V3,Al-SDQPV et ANAMSO (,"colza, p",colza,.2) En butinant de f,6979-6987,"type=N_PUN_N, form=colza, p"
1,07_BSV_GC_16042013_cle83b3df,ançon de la tige du,colza et méligèthes,colza,peut être envisagée,1475-1494,"type=N_KON_N, form=colza et méligèthes"
2,08_BSV_ZNA_080714_cle0bae61,", fétuques, pâturin",annuel et ray-grass,ray-grass,. Le champignon se d,3848-3867,"type=N_KON_N, form=annuel et ray-grass"


In [62]:
df = pd.read_csv("../output/frenchCropUsage_20210525/alea/conjunctions.csv", delimiter="\t", encoding='utf-8')
df[:3]

,bsv,context before,conjunction,match,context after,location,features
0,05_GC_Als_cle8c3f61,écocement. Comme en,"maïs, les",maïs,attaques d'oscinies,1006-1015,"type=N_PUN_N, form=maïs, les"
1,05bsv_jevi_20190502_cle8f22b6,"ine. Espaces verts,",parc et jardins,"parc, jardins",d'amateurs. Observa,2754-2769,"type=N_KON_N, form=parc et jardins"
2,05bsv_jevi_20190502_cle8f22b6,t Gesvres et région,"lavalloise, jardins",jardins,d'amateurs. Observa,4092-4111,"type=N_PUN_N, form=lavalloise, jardins"


## Extracted conjunctions 

In [63]:
articles = ["le", "la", "les", "un", "une"]
phenologie = [ "application", "applications","activité", "apparition","arrachage", "arrachages", "avortement", "attaques",    
             "base", 
             "croissance",  "contamination", "contaminations","compacité","capacité", "cycle", "cycles","couleur", "captures", "cesse",
             "dynamique", "disponibilité", "développement", "dissémination","dépérissements","début",
             "état",  "enracinement", "éclatement", "éclatements","etablissement", "etablissements","effets",
             "formation","fin","fauchage","fermeture","gestion",
             "grossissement", 
             "herse",
             "infestation","installation","implication", "inoculum",
             "levée", "levées",
             "multiplications","mileos",
             "nuisibilité","nombre",
             "présence","perte", "pertes", "pollution", "phase","production","pousse","période","proximité", "ponte","piégeages",
             "réalisation","résistance", "redémarrage", "repousse","repousses", "rangs","rotation", "rotations","rendement",
             "sensibilité",  "stade", "stades", "sporulation","seuil", 
             "tallage","taille", "touffes",
             "qualité",
             "véraison", "variété","végétation"]

superficies = ["abords",
               "bordure","bordures","bulbe",
               "cocons",
               "grains", "grain","grappe", "grappes","graine", "graines",
               "feuilles", "feuille",
               "lots", "lot",
               "niveaux", "niveau",
               "parcelle", "parcelles","part", "parts",
               "reste", "restes","résidus",
               "stock","sol",
               "tige", "tiges","trou", "trous",
               "zone", "zones",
              ]

bioagresseurs = ["aleurode","altise",
                 "bactéries","bactérie","botrytis",
                 "charançon", "chenille",
                 "hôtes",
                 "larve", "larves",
                 "mouche", "mouches","mineuse",
                 "nezara",
                 "punaise","pucerons","piéride",
                 "ravageurs",
                "criquets"]

maladies = ["maladie", "maladies","mildiou", "minitans", "oïdium", "septoriose"]

ambiguous_words = ["auxiliaire", "auxiliaires","avril","approche","agriculture",
             "cave", "culture","cultures","contraire","coulures", "coteaux","culture", "cultures","condition", "conditions","compte", "comptes","coopérative","champlitte",
             "dégât", "dégâts","d'agriculture",
             "effectifs","effet", "effets",
             "fleur", "fleurs", "fruits",
             "gel",
             "heure", "heures",
             "interculture","inflorescences","indice",
             "jour", "jours","jura",
             "maison","modélisation", "modélisations","maraîchage", "mai",
            "labastide",
             "noire",
             "oiseau", "oiseaux", "orages",
             "photo","producteurs", "plupart","pluies",
             "région", "régions","risque", "risques",
             "société","semaine","sein", "semaines","station", "stations","secteur", "secteurs","symptôme", "symptômes","situation", "situations", "sol", "sols",
             "techniciens",  "teigne", "tissu", "tissus", "temps","tarn",
             "vallée",  "viticulture", "vigueur", "vue", "vol", "vols","végétaux", "viticulture", 
             "usage", "usages",
            ]

filters = superficies + phenologie + bioagresseurs + maladies + ambiguous_words + articles

empty_words = ["lorsqu'elles",
              "c'est",
              "d'autres",
              "sra",
              "mi",       
              "d'hiver",             
              "des", "de", "du", "surveillerchou", "p", "ceux-ci"]

In [66]:
values = df["conjunction"].values.tolist()
conjunctions_list = []
for line in values:
    line = line.lower()
    line = line.replace(' et ', ',')
    line = line.replace('(', ',')
    line = line.replace(')', ',')
    line = line.replace("l'", '')
    lines = line.split(',')
    
    lines_clean = []
    for l in lines:
        le = l.split()
        le = [word for word in le if word not in filters]
        #le = [word for word in le if word not in stages]
        le = [word for word in le if len(word) != 1]
        le = [word for word in le if not word.isdigit()]
        lines_clean.append(" ".join(le))
        
        
    #lines_clean = [word for word in lines_clean if word not in ambiguous_words]
    lines_clean = [word for word in lines_clean if word not in empty_words]
    
    words = []
    for word in lines_clean:
        if word.startswith("des "):
            word = word.replace("des", "")
        if word.startswith("de "):
            word = word.replace("de", "")
        if word.startswith("du "):
            word = word.replace("du", "")
        words.append(word)

    final = [word.strip() for word in words]
    final = [word for word in final if word != ""]
    conjunctions_list.append(final)
len(conjunctions_list)


399

## FCU terms

In [67]:
matches = df["match"].values.tolist()

fcu_list = []
for line in matches:
    line = line.lower()
    lines = line.split(', ')
    lines = [word.strip() for word in lines]
    fcu_list.append(lines)
fcu_list

[['maïs'],
 ['parc', 'jardins'],
 ['jardins'],
 ['jardins'],
 ['betterave', 'épinard', 'poirée'],
 ['jardin'],
 ['vergers'],
 ['vignes'],
 ['prunier', 'cerisier', 'abricotier'],
 ['champignons'],
 ['champignons'],
 ['poinsettias'],
 ['géranium', 'chrysanthème'],
 ['jardins'],
 ['jardins'],
 ['pins'],
 ['parcs'],
 ['pois', 'féverole'],
 ['carotte', 'céleri', 'persil'],
 ['oignon'],
 ['navet'],
 ['persil'],
 ['carotte', 'céleri', 'persil'],
 ['laitue'],
 ['blé'],
 ['blé'],
 ['orge', 'escourgeon'],
 ['colza'],
 ['banane'],
 ['banane'],
 ['banane'],
 ['jardins'],
 ['banane'],
 ['banane'],
 ['bois'],
 ['champignon'],
 ['banane'],
 ['banane'],
 ['banane'],
 ['patates'],
 ['poire'],
 ['prunes', 'cerises'],
 ['pommes', 'poires'],
 ['pommes', 'poires'],
 ['pommes', 'poires'],
 ['pommes', 'poires'],
 ['arboriculture', 'maraîchage', 'pépinière'],
 ['arboriculture', 'maraîchage', 'pépinière'],
 ['horticulture'],
 ['champignons'],
 ['pommes'],
 ['champignons'],
 ['abricots', 'pêches'],
 ['pêches', 

## Propositions

In [68]:
def Diff(li1, li2):
    return list(set(li1) - set(li2)) 

diffs = []
for i, line in enumerate(conjunctions_list):
    diff = Diff(line, fcu_list[i])
    diff = ",\n".join(str(x) for x in diff)
    diffs.append(diff)

## Add propositions 

In [69]:
df.insert(4,'proposition',diffs)
df[:3]

,bsv,context before,conjunction,match,proposition,context after,location,features
0,05_GC_Als_cle8c3f61,écocement. Comme en,"maïs, les",maïs,,attaques d'oscinies,1006-1015,"type=N_PUN_N, form=maïs, les"
1,05bsv_jevi_20190502_cle8f22b6,"ine. Espaces verts,",parc et jardins,"parc, jardins",,d'amateurs. Observa,2754-2769,"type=N_KON_N, form=parc et jardins"
2,05bsv_jevi_20190502_cle8f22b6,t Gesvres et région,"lavalloise, jardins",jardins,lavalloise,d'amateurs. Observa,4092-4111,"type=N_PUN_N, form=lavalloise, jardins"


In [70]:
df.to_csv("../output/frenchCropUsage_20210525/alea/propositions_for_fcu.csv", sep = '\t', encoding = 'utf-8', index=True)